In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama3-70b-8192"
)

response = llm.invoke("The first person to land on the moon was ...")
print(response.content)

Neil Armstrong! On July 20, 1969, Neil Armstrong became the first person to set foot on the moon, famously declaring "That's one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle onto the moon's surface.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.adidas-group.com/job/Paris-Manager-analyses-commerciales-&-insights-%28HFD%29/1232764601/?feedId=301201&utm_source=j2w")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.















Manager analyses commerciales & insights (H/F/D) Job Details | adidas

































                    By continuing to use and navigate this website, you are agreeing to the use of cookies.
                
            

                Accept

                Close




Press Tab to Move to Skip to Content Link
Skip to main content








Prepare for interview
Search Jobs
SEE CATEGORIES




















Search by Keyword




Search by Location







                                 
                            















Prepare for interview
Search Jobs
SEE CATEGORIES









View Profile




Employee Login






















Search by Keyword




Search by Location






Show More Options





Loading...







                                            Team:
                                        


All





                                            Location
                                        


All





                      

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTIONS
    The scraped text from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm
result = chain_extract.invoke(input={'page_data': page_data})
print(type(result.content))
result

<class 'str'>


AIMessage(content='Here is the extracted job posting in JSON format:\n\n```\n{\n    "role": "Manager analyses commerciales & insights (H/F/D)",\n    "experience": "at least 4 years of experience in data analytics / business intelligence",\n    "skills": [\n        "Excel",\n        "Power BI",\n        "SQL",\n        "Python",\n        "Alteryx"\n    ],\n    "description": "Analyser les KPI Sales sur tous les canaux (Wholesale & Direct-to-Consumer) de manière très régulière et proactive. Être le garant de la connaissance de la performance commerciale de la marque en France, en support de l’Analyste commercial senior, et partager des insights clairs et actionnables aux équipes commerciales via des présentations impactantes (audience Managers, Senior Managers et Top Management)."\n}\n```', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 2028, 'total_tokens': 2208, 'completion_time': 0.301961071, 'prompt_time': 0.154248188, 'queue_time'

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(result.content) 
json_res

{'role': 'Manager analyses commerciales & insights (H/F/D)',
 'experience': 'at least 4 years of experience in data analytics / business intelligence',
 'skills': ['Excel', 'Power BI', 'SQL', 'Python', 'Alteryx'],
 'description': 'Analyser les KPI Sales sur tous les canaux (Wholesale & Direct-to-Consumer) de manière très régulière et proactive. Être le garant de la connaissance de la performance commerciale de la marque en France, en support de l’Analyste commercial senior, et partager des insights clairs et actionnables aux équipes commerciales via des présentations impactantes (audience Managers, Senior Managers et Top Management).'}

In [7]:
type(json_res)

dict

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [10]:
links = collection.query(query_texts=['Experience in Python', 'Expertise in React Native'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [11]:
job = json_res
job['skills']

['Excel', 'Power BI', 'SQL', 'Python', 'Alteryx']

In [12]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}]]

In [14]:
prompt_email = PromptTemplate.from_template(
    '''
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Kirtirajsinh Parmar, a business development executive at NexaCore Solutions. NexaCore is an AI and Software Consulting company dedicated to faciliating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimisation, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of NexaCore
    in fufilling their needs.
    Also add the most relevant ones from the following links to showcase NexaCore's portfolio: {link_list}
    Remember that you are Kirtirajsinh Parmar, BDE at NexaCore Solutions.
    ### EMAIL (NO PREAMBLE)
    
    '''
    )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Data-Driven Insights with NexaCore Solutions

Dear Hiring Manager,

I came across the job posting for a Manager of Commercial Analyses and Insights, and I was impressed by the emphasis on leveraging data analytics to drive business growth. As a Business Development Executive at NexaCore Solutions, I believe our expertise in AI and software consulting can help your organization unlock the full potential of its commercial data.

With our proficiency in Excel, Power BI, SQL, Python, and Alteryx, we can help you streamline your data analysis process, providing actionable insights to support your commercial teams. Our team of experts can develop tailored solutions to integrate your data from various channels, including Wholesale and Direct-to-Consumer, and create impactful presentations to inform business decisions.

At NexaCore, we have a proven track record of empowering enterprises with scalable and efficient solutions. Our portfolio includes successful projects in data a